# Recommendation Systems for movies

![](http://labs.criteo.com/wp-content/uploads/2017/08/CustomersWhoBought3.jpg)

Two MovieLens datasets are used here. (link : https://www.kaggle.com/rounakbanik/the-movies-dataset/data)

* **The Full Dataset:** Consists of 26,000,000 ratings and 750,000 tag applications applied to 45,000 movies by 270,000 users. Includes tag genome data with 12 million relevance scores across 1,100 tags.
* **The Small Dataset:** Comprises of 100,000 ratings and 1,300 tag applications applied to 9,000 movies by 700 users.

We will build our Simple Recommender using movies from the *Full Dataset* whereas all personalized recommender systems will make use of the small dataset (due to the computing power I possess being very limited). As a first step, let us build our simple recommender system.

In [102]:
#importing required libraries 
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None

from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD


import warnings; warnings.simplefilter('ignore')


## Simple Popularity recommendation model

The model simply do implement generalize recommendation to all user based on popularity of movie. The logic behind the model is average audience probably pick or like movies which is critically acclaimed and most popular.This model does not give personalized recommendations based on the user. 

Procedure is as follow.

- First sort movies based on ratings and popularity
- Display the top movies of list. 

** ---- added step ------- ** <br> 
also used genre argument for the top movies of a particular genre. 

In [103]:
m_df = pd.read_csv('data/movies_metadata.csv')
# m_df.head()

In [104]:
m_df['genres'] = m_df['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] 
                                                                       if isinstance(x, list) else [])


#### IMDB's *weighted rating* formula 

Mathematically, it is represented as follows:

Weighted Rating (WR) = $(\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$

where,
* *v* is the number of votes for the movie
* *m* is the minimum votes required to be listed in the chart
* *R* is the average rating of the movie
* *C* is the mean vote across the whole report

Now, determine an cutoff value for *m*, the minimum votes required to be listed in the chart. 

in this model used cutoff is **95th percentile** that means for a movie to feature in the top N must have more votes than at least 95% of the movies in the list.

#### Calculate C

In [105]:
# Claculation of c 
vote_counts = m_df[m_df['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = m_df[m_df['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

#### Calculate m 

In [106]:
## Claculation of m
m = vote_counts.quantile(0.95)
m

434.0

In [107]:
# extracting release year from release_date
m_df['year'] = pd.to_datetime(m_df['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] 
                                                                           if x != np.nan else np.nan)

#### Getting qualified movies (cutoff: 95%)

In [108]:
col_list = ['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']
qualified = m_df[(m_df['vote_count'] >= m) 
                 & (m_df['vote_count'].notnull()) 
                 & (m_df['vote_average'].notnull())][col_list]

# converting vote_count and vote_average colums to integer
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

Findings as follow:
1. for consideration in the chart, a movie has to have at least **434 votes**. 
2. the average rating for a movie is **5.244** on a scale of 10. 
3. **2274** Movies qualify to be on the chart.

### Calculating weighted ratings and making top 250 chart

In [109]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [110]:
qualified['weighted_rating'] = qualified.apply(weighted_rating, axis=1)

In [111]:
qualified = qualified.sort_values('weighted_rating', ascending=False).head(250)

### Top 10 Movies

In [113]:
temp = m_df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
temp.name = 'genre'
mgen_df = m_df.drop('genres', axis=1).join(temp)

In [114]:
def make_toplist(genre, percentile=0.85):
    df = mgen_df[mgen_df['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    col_list = ['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genre']
    qualified = df[(df['vote_count'] >= m) 
                   & (df['vote_count'].notnull()) 
                   & (df['vote_average'].notnull())][col_list]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['weighted_rating'] = qualified.apply(lambda x: 
                                                   (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), 
                                                   axis=1)
    qualified = qualified.sort_values('weighted_rating', ascending=False).head(250)
    
    return qualified

# Top 10 movies in a genre


In [115]:
genre = input("Enter the genre: ")

Enter the genre: Action


In [116]:
make_toplist(genre).head(10)

,title,year,vote_count,vote_average,popularity,genre,weighted_rating
15480,Inception,2010,14075,8,29.1081,Action,7.955099
12481,The Dark Knight,2008,12269,8,123.167,Action,7.948610
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,Action,7.929579
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,Action,7.924031
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,Action,7.918382
256,Star Wars,1977,6778,8,42.1497,Action,7.908327
1154,The Empire Strikes Back,1980,5998,8,19.471,Action,7.896841
4135,Scarface,1983,3017,8,11.2997,Action,7.802046
9430,Oldboy,2003,2000,8,10.6169,Action,7.711649
1910,Seven Samurai,1954,892,8,15.0178,Action,7.426145
